# # Dask tutorial 

#### Using Dask on HPC 
* [Dask joblib implementation](https://ml.dask.org/joblib.html)

In [1]:
from dask.distributed import Client
import joblib

client = Client(processes=False)             # create local cluster
# client = Client("scheduler-address:8786")  # or connect to remote cluster

In [2]:
client

Client Scheduler: inproc://172.17.0.5/3147/1 Dashboard: http://172.17.0.5:8787/status,Cluster Workers: 1 Cores: 80 Memory: 540.94 GB


In [7]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import time as time 

digits = load_digits()

param_space = {
    'C': np.logspace(-6, 6, 13),
    'gamma': np.logspace(-8, 8, 17),
    'tol': np.logspace(-4, -1, 4),
    'class_weight': [None, 'balanced'],
}

model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, n_jobs=1, cv=5, n_iter=100, verbose=1)


start = time.time() 
search.fit(digits.data, digits.target)
#with joblib.parallel_backend('dask'):
#    search.fit(digits.data, digits.target)
end = time.time() - start 
print('Time take: {:0.3f} secs'.format(end))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Time take: 163.661 secs


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.7min finished


In [10]:
start = time.time() 
with joblib.parallel_backend('dask'):
    search.fit(digits.data, digits.target)
end = time.time() - start 
print('Time take: {:0.3f} secs'.format(end))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 80 concurrent workers.
[Parallel(n_jobs=1)]: Done  42 tasks      | elapsed:    1.5s
[Parallel(n_jobs=1)]: Done 292 tasks      | elapsed:    4.5s


Time take: 5.733 secs


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    5.6s finished
Exception ignored in: <function _WeakKeyDictionary.__setitem__.<locals>.on_destroy at 0x7fa797efe200>
Traceback (most recent call last):
  File "/opt/conda/envs/rapids/lib/python3.7/site-packages/joblib/_dask.py", line 74, in on_destroy
    del self._data[key]
KeyError: (140357764641536,)
